## 作業
礙於不是所有同學都有 GPU ，這邊的範例使用的是簡化版本的 ResNet，確保所有同學都能夠順利訓練!


最後一天的作業請閱讀這篇非常詳盡的[文章](https://blog.gtwang.org/programming/keras-resnet-50-pre-trained-model-build-dogs-cats-image-classification-system/)，基本上已經涵蓋了所有訓練　CNN 常用的技巧，請使用所有學過的訓練技巧，盡可能地提高 Cifar-10 的 test data 準確率，截圖你最佳的結果並上傳來完成最後一次的作業吧!

另外這些技巧在 Kaggle 上也會被許多人使用，更有人會開發一些新的技巧，例如使把預訓練在 ImageNet 上的模型當成 feature extractor 後，再拿擷取出的特徵重新訓練新的模型，這些技巧再進階的課程我們會在提到，有興趣的同學也可以[參考](https://www.kaggle.com/insaff/img-feature-extraction-with-pretrained-resnet)

In [2]:
import keras
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [7]:
from keras.datasets import cifar10
from keras.applications.resnet50 import ResNet50
from keras.layers import Flatten, Dense, Dropout

In [4]:
num_classes = 10
epochs = 10
batch_size = 32

In [11]:
# 讀取 Cifar-10 資料集
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [16]:
# 以訓練好的 ResNet50 為基礎來建立模型，
# 捨棄 ResNet50 頂層的 fully connected layers
net = ResNet50(include_top=False, input_shape=x_train.shape[1:])
x = net.output
x = Flatten()(x)
x = Dropout(0.5)(x)

output_layer = Dense(units=num_classes, activation='softmax', name = 'output_layer')(x)

model = keras.models.Model(inputs = net.input, outputs = output_layer)
print(model.summary())

94773248/94765736 [==============================] - 0s 0us/step
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 38, 38, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 16, 16, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 16, 16, 64)   256         conv1_conv[0][0]                 
_____________________________

In [17]:
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer=keras.optimizers.Adam(learning_rate=1e-5))

In [18]:
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(x_test, y_test), verbose=1)

Epoch 1/10
1563/1563 [==============================] - 105s 43ms/step - loss: 3.8369 - accuracy: 0.1595 - val_loss: 2.1796 - val_accuracy: 0.3933
Epoch 2/10
1563/1563 [==============================] - 67s 43ms/step - loss: 2.1608 - accuracy: 0.3432 - val_loss: 1.6193 - val_accuracy: 0.5107
Epoch 3/10
1563/1563 [==============================] - 68s 43ms/step - loss: 1.5914 - accuracy: 0.4734 - val_loss: 1.2848 - val_accuracy: 0.5921
Epoch 4/10
1563/1563 [==============================] - 67s 43ms/step - loss: 1.3172 - accuracy: 0.5559 - val_loss: 1.1178 - val_accuracy: 0.6445
Epoch 5/10
1563/1563 [==============================] - 68s 43ms/step - loss: 1.1214 - accuracy: 0.6176 - val_loss: 1.0107 - val_accuracy: 0.6798
Epoch 6/10
1563/1563 [==============================] - 67s 43ms/step - loss: 0.9798 - accuracy: 0.6673 - val_loss: 0.9457 - val_accuracy: 0.7050
Epoch 7/10
1563/1563 [==============================] - 68s 43ms/step - loss: 0.8737 - accuracy: 0.7003 - val_loss: 0.8875 

In [19]:
model.evaluate(x_test, y_test)

313/313 [==============================] - 4s 14ms/step - loss: 0.8065 - accuracy: 0.7592


[0.8065150380134583, 0.7591999769210815]